In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding,Dense,LSTM,Bidirectional
from tensorflow.keras.models import Sequential
import pandas
import nltk
from nltk.stem import WordNetLemmatizer
import re

In [2]:
maxlength=100
stopwords=nltk.corpus.stopwords.words("english")
dataset=pandas.read_csv("data/sentiment_analysis.csv")
x=dataset["text"]
y=dataset["label"]
tokenizer=Tokenizer(oov_token="UNK")
lemmatizer=WordNetLemmatizer()

In [3]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [4]:
tokenizer.fit_on_texts(x_train)
vocabulary_size=len(tokenizer.word_index)+1

In [5]:
def texts_to_sequences(texts):
    array=[]
    for text in texts:
        text=re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?","",text)
        tokens=nltk.word_tokenize(text)
        tokens=[lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stopwords]
        array.append(" ".join(tokens))
    texts=tokenizer.texts_to_sequences(array)
    return pad_sequences(texts,maxlength,truncating="post")

In [6]:
x_train=texts_to_sequences(x_train)
x_test=texts_to_sequences(x_test)

In [7]:
model=Sequential([
    Embedding(input_dim=vocabulary_size,output_dim=32),
    Bidirectional(LSTM(32)),
    Dense(1,"sigmoid")
])
model.compile("adam","binary_crossentropy","accuracy")
model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test))

Epoch 1/2
1172/1172 [==============================] - 179s 146ms/step - loss: 0.4701 - accuracy: 0.7604 - val_loss: 0.3171 - val_accuracy: 0.8660
Epoch 2/2
1172/1172 [==============================] - 164s 140ms/step - loss: 0.1812 - accuracy: 0.9333 - val_loss: 0.3253 - val_accuracy: 0.8672


In [8]:
texts=["I love this movie it is great","I hate this movie it is terrible"]
texts=texts_to_sequences(texts)
model.predict(texts)

array([[0.83972394],
       [0.17692232]], dtype=float32)